In [11]:
import json
import websocket
import pandas as pd

# Creating a connection to serv


In [12]:
API_Key = "VKBSmOKljPJF673UkPloF5mZ5X4DruNUvWOOZ6zo8D74rQMtpA0blnPJsEbVTTq2"
Secret_Key = "bdVCIsdAwqSxzIgbRPPuBHr5n0KMgOxNw9JFFr00YUvKmZgGCr9vSdp4nhsYS59O"

In [13]:
from binance.client import Client
import pandas as pd

In [14]:
client = Client(api_key = API_Key, api_secret = Secret_Key, tld="com")

In [15]:
client.ping()

{}

In [16]:
account = client.get_account()

In [17]:
balances_df = pd.DataFrame(account["balances"])
balances_df

,asset,free,locked
0,BTC,0.03206461,0.00000000
1,LTC,0.00000000,0.00000000
2,ETH,0.41258079,0.22410000
3,NEO,0.00000000,0.00000000
4,BNB,0.99748603,0.00000000
...,...,...,...
559,JTO,0.00000000,0.00000000
560,ACE,0.00000000,0.00000000
561,1000SATS,0.00,0.00
562,BONK,0.00,0.00


#Check binance trade fees


In [18]:
client.get_trade_fee(symbol="BTCUSDT")

[{'symbol': 'BTCUSDT', 'makerCommission': '0.001', 'takerCommission': '0.001'}]

In [19]:
client.get_trade_fee(symbol="BUSDUSDT")

[{'symbol': 'BUSDUSDT', 'makerCommission': '0', 'takerCommission': '0'}]

In [20]:
client.get_symbol_info(symbol= "BTCUSDT")

{'symbol': 'BTCUSDT',
 'status': 'TRADING',
 'baseAsset': 'BTC',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'cancelReplaceAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.01000000',
   'maxPrice': '1000000.00000000',
   'tickSize': '0.01000000'},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.00001000',
   'maxQty': '9000.00000000',
   'stepSize': '0.00001000'},
  {'filterType': 'ICEBERG_PARTS', 'limit': 10},
  {'filterType': 'MARKET_LOT_SIZE',
   'minQty': '0.00000000',
   'maxQty': '122.73336966',
   'stepSize': '0.00000000'},
  {'filterType': 'TRAILING_DELTA',
   'minTra

# Get Live prices

In [21]:
client.get_symbol_ticker(symbol = "BTCUSDT") # current price for one symbol

{'symbol': 'BTCUSDT', 'price': '41996.03000000'}

In [22]:
client.get_avg_price(symbol="BTCUSDT") # recent avg price for last 5 mins 

{'mins': 5, 'price': '41990.40263832', 'closeTime': 1703912379665}

In [23]:
float(client.get_symbol_ticker(symbol = "BTCUSDT")["price"])

41996.02

In [24]:
last24 = client.get_ticker(symbol= "BTCUSDT")
last24


{'symbol': 'BTCUSDT',
 'priceChange': '-665.95000000',
 'priceChangePercent': '-1.561',
 'weightedAvgPrice': '42272.11646243',
 'prevClosePrice': '42661.97000000',
 'lastPrice': '41996.02000000',
 'lastQty': '0.00437000',
 'bidPrice': '41996.02000000',
 'bidQty': '1.97726000',
 'askPrice': '41996.03000000',
 'askQty': '3.72206000',
 'openPrice': '42661.97000000',
 'highPrice': '43111.00000000',
 'lowPrice': '41300.00000000',
 'volume': '38487.51539000',
 'quoteVolume': '1626948732.91579120',
 'openTime': 1703825982344,
 'closeTime': 1703912382344,
 'firstId': 3341558538,
 'lastId': 3342975980,
 'count': 1417443}

In [25]:
day_price_change = last24['priceChange']
day_price_change

'-665.95000000'

# Getting Historical Data (OHLC & Volume)

In [26]:
timestamp = client._get_earliest_valid_timestamp(symbol = "BTCUSDT", interval = "1d")
timestamp

1502928000000

In [27]:
pd.to_datetime(timestamp, unit="ms") # earlist data from binance

Timestamp('2017-08-17 00:00:00')

In [28]:
def get_history(symbol, interval, start, end= None):
    bars = client.get_historical_klines(symbol = symbol,
                                    interval = interval, start_str = start, end_str = end, limit=1000)
    historicalPrices = pd.DataFrame(bars)
    historicalPrices["Date"] = pd.to_datetime(historicalPrices.iloc[:,0], unit= "ms") 
    historicalPrices.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Clos Time",
                                "Quote Asset Volume", "Number of Trades", "Taker Buy Base Asset Volume",
                                "Taker Buy Quote Asset Volume","Ignore", "Date"]
    historicalPrices = historicalPrices[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    historicalPrices.set_index("Date", inplace=True)
    for column in historicalPrices.columns:
        historicalPrices[column] = pd.to_numeric(historicalPrices[column], errors = "coerce")
    
    return historicalPrices

In [29]:
df = get_history(symbol="BTCUSDT", interval="1d", start=timestamp)
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...
2023-12-26,43576.12,43592.68,41637.60,42508.93,41010.042820
2023-12-27,42508.93,43677.00,42098.69,43428.85,36191.211360
2023-12-28,43428.86,43787.57,42241.79,42563.76,35150.524850


# Most recent (last 2 hours) Intraday Data (1m)


In [30]:
from datetime import datetime, timedelta

In [31]:
now = datetime.now()
now

datetime.datetime(2023, 12, 30, 5, 59, 45, 740658)

In [32]:
two_hours_before = now - timedelta(hours=2)
str(two_hours_before)

'2023-12-30 03:59:45.740658'

In [33]:
last_2h_1m_df = get_history(symbol = "BTCUSDT", interval = "1m", start=str(two_hours_before))
last_2h_1m_df

,Open,High,Low,Close,Volume
Date,,,,,
2023-12-30 04:00:00,41972.81,41972.82,41972.81,41972.81,3.69742
2023-12-30 04:01:00,41972.82,41979.56,41969.49,41979.55,13.00284
2023-12-30 04:02:00,41979.55,41994.18,41979.55,41982.61,12.49983
2023-12-30 04:03:00,41982.61,41996.21,41982.60,41996.21,6.13729
2023-12-30 04:04:00,41996.20,42000.00,41996.20,42000.00,9.25251
2023-12-30 04:05:00,41999.99,42040.00,41999.99,42039.99,15.99583
2023-12-30 04:06:00,42040.00,42051.68,42039.99,42047.46,8.67184
2023-12-30 04:07:00,42047.47,42062.92,42044.96,42054.44,13.96747
2023-12-30 04:08:00,42054.44,42054.45,42032.98,42032.99,11.62693


# Option 1: ThreadedWebsocketManager (only executable in scripts!)

In [34]:
from binance import ThreadedWebsocketManager
import time

In [35]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Coplete"])
df

,Open,High,Low,Close,Volume,Coplete


In [36]:
def stream_candles(msg):
    
    #extract required candle items
    event_time = pd.to_datetime(msg["E"], unit="ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit="ms")
    first      = float(msg["k"]["o"])
    high      = float(msg["k"]["h"])
    low      = float(msg["k"]["l"])
    close      = float(msg["k"]["c"])
    volume      = float(msg["k"]["v"])
    complete      = msg["k"]["x"]
    
    # print out
    print("Time: {} | Price: {}".format(event_time, close))
    
    # feed df (add a new bar / update)
    df.loc[start_time] = [first,high,low,close,volume,complete]
    

In [37]:
# initialize the start WebSocket

twm = ThreadedWebsocketManager(tld="us")
twm.start()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\matei\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\matei\anaconda3\lib\site-packages\binance\threaded_stream.py", line 59, in run
    self._loop.run_until_complete(self.socket_listener())
  File "C:\Users\matei\anaconda3\lib\asyncio\base_events.py", line 623, in run_until_complete
    self._check_running()
  File "C:\Users\matei\anaconda3\lib\asyncio\base_events.py", line 583, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running


In [38]:
twm.start_symbol_miniticker_socket(callback = stream_data, symbol = "BTCUSDT")

C:\Users\matei\anaconda3\lib\threading.py:982: RuntimeWarning: coroutine 'ThreadedApiManager.socket_listener' was never awaited
  self._invoke_excepthook(self)


NameError: name 'stream_data' is not defined

In [ ]:
While True:
    time.sleep(20)
    twm.stop()
    break

# Option 2: BinanceSocketManager (executable in Jupyter!)

In [ ]:
import asyncio
from binance import AsyncClient, BinanceSocketManager

In [ ]:
def stream_data(msg):
    print(msg)

In [ ]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.symbol_miniticker_socket(symbol="BTCUSDT")

    async with ts as tscm:  # Add a colon here
        res = await tscm.recv()
        stream_data(res)

    await client.close_connection()

In [ ]:
n = 10


In [ ]:
await main()

# Creating a Test Order

In [44]:
from binance.client import Client
import pandas as pd 

In [ ]:
# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(symbol="BTCUSDT", side = "BUY", type="MARKET", quantity=0.1)

In [40]:
## Paper Trading in the Spot Testnet

# Paper Trading with the Spot Testnet API

In [41]:
api_key = "VxuqPa3p6RrAN8wBxSP7BFcFr1xyA6Wc9FrWdOu6lFMANIlr5UhekZdep67NBgNJ"
secret_key = "EgB2E9eq55Kkjp6SJTpU4GK0Yp0SMqNyLdoHyC9cdbakF6BFTGonfloDADbGJYVb"

In [42]:
client = Client(api_key = api_key, api_secret = secret_key, tld="com", testnet=True)

In [48]:
test_account = client.get_account()

In [52]:
df = pd.DataFrame(test_account["balances"])
df

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,LTC,6.00000000,0.00000000
3,BNB,2.00000000,0.00000000
4,USDT,10000.00000000,0.00000000
...,...,...,...
373,ORDI,10.00000000,0.00000000
374,BEAMX,18446.00000000,0.00000000
375,VIC,508.00000000,0.00000000
376,BLUR,996.00000000,0.00000000


In [62]:
client.get_asset_balance(asset="BTC")

{'asset': 'BTC', 'free': '1.10000000', 'locked': '0.00000000'}

In [61]:
client.get_asset_balance(asset="USDT")

{'asset': 'USDT', 'free': '5792.04785980', 'locked': '0.00000000'}

# Place a buy Order

In [59]:
order = client.create_order(symbol="BTCUSDT", side= "SELL", type="MARKET", quantity=0.1)

In [60]:
order

{'symbol': 'BTCUSDT',
 'orderId': 13831141,
 'orderListId': -1,
 'clientOrderId': 'LqIx5AnRni47paNdBYt8fY',
 'transactTime': 1703912774419,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '4205.86797400',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1703912774419,
 'fills': [{'price': '42061.99000000',
   'qty': '0.00690000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2314750},
  {'price': '42061.95000000',
   'qty': '0.01035000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2314751},
  {'price': '42061.30000000',
   'qty': '0.00666000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2314752},
  {'price': '42061.04000000',
   'qty': '0.00654000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2314753},
  {'price': '42060.03000000',
   'qty': '0.00975000',
   'commi

In [63]:
df = pd.DataFrame(order["fills"])
df

,price,qty,commission,commissionAsset,tradeId
0,42061.99000000,0.00690000,0.00000000,USDT,2314750
1,42061.95000000,0.01035000,0.00000000,USDT,2314751
2,42061.30000000,0.00666000,0.00000000,USDT,2314752
3,42061.04000000,0.00654000,0.00000000,USDT,2314753
4,42060.03000000,0.00975000,0.00000000,USDT,2314754
5,42058.86000000,0.01166000,0.00000000,USDT,2314755
6,42058.67000000,0.01130000,0.00000000,USDT,2314756
7,42056.95000000,0.01154000,0.00000000,USDT,2314757
8,42056.90000000,0.01154000,0.00000000,USDT,2314758
9,42056.88000000,0.00856000,0.00000000,USDT,2314759


In [69]:
num_columns = ["price", "qty", "commission"]

In [70]:
for column in num_columns:
    df[column] = pd.to_numeric(df[column], errors="coerce")

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            11 non-null     float64
 1   qty              11 non-null     float64
 2   commission       11 non-null     float64
 3   commissionAsset  11 non-null     object 
 4   tradeId          11 non-null     int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 568.0+ bytes


In [72]:
df.qty.sum()

0.09999999999999999

In [74]:
df.price.mul(df.qty).sum()

4205.867974

# Place Limit Order

In [88]:
client.get_symbol_ticker(symbol="BTCUSDT")

{'symbol': 'BTCUSDT', 'price': '42060.28000000'}

In [79]:
order1 = client.create_order(symbol="BTCUSDT", side="BUY", type="LIMIT",
                             quantity=0.1, timeInForce="GTC", price = 42000)

In [82]:
order1

{'symbol': 'BTCUSDT',
 'orderId': 13835175,
 'orderListId': -1,
 'clientOrderId': 'xj4t2NyauVLSA5nnkfGfCL',
 'transactTime': 1703913383983,
 'price': '42000.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'workingTime': 1703913383983,
 'fills': [],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [84]:
id_buy_limit1 = order1["orderId"]
id_buy_limit1

13835175

In [87]:
client.get_order(symbol = "BTCUSDT", orderId = id_buy_limit1)

{'symbol': 'BTCUSDT',
 'orderId': 13835175,
 'orderListId': -1,
 'clientOrderId': 'xj4t2NyauVLSA5nnkfGfCL',
 'price': '42000.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1703913383983,
 'updateTime': 1703913383983,
 'isWorking': True,
 'workingTime': 1703913383983,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [91]:
order2 = client.create_order(symbol="BTCUSDT", side="SELL", type="LIMIT",
                             quantity=0.1, timeInForce="GTC", price = 43000)

In [92]:
id_sell_limit1 = order2["orderId"]


In [100]:
open_orders = client.get_open_orders(symbol="BTCUSDT")
pd.DataFrame(open_orders)

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCUSDT,13835175,-1,xj4t2NyauVLSA5nnkfGfCL,42000.00000000,0.10000000,0.00000000,0.00000000,NEW,GTC,LIMIT,BUY,0.00000000,0.00000000,1703913383983,1703913383983,True,1703913383983,0.00000000,EXPIRE_MAKER


In [99]:
result = client.cancel_order(symbol = "BTCUSDT", orderId = id_sell_limit1)

In [101]:
all_orders = client.get_all_orders(symbol="BTCUSDT")
pd.DataFrame(all_orders)

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCUSDT,13830784,-1,AQvWZ1K7Cjq0jRhmmGUfSZ,0.00000000,0.10000000,0.10000000,4206.79569810,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1703912730605,1703912730605,True,1703912730605,0.00000000,EXPIRE_MAKER
1,BTCUSDT,13830813,-1,5iYLLGrLfsqRgs4l4dJmT2,0.00000000,0.10000000,0.10000000,4207.02441610,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1703912734008,1703912734008,True,1703912734008,0.00000000,EXPIRE_MAKER
2,BTCUSDT,13831141,-1,LqIx5AnRni47paNdBYt8fY,0.00000000,0.10000000,0.10000000,4205.86797400,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1703912774419,1703912774419,True,1703912774419,0.00000000,EXPIRE_MAKER
3,BTCUSDT,13835175,-1,xj4t2NyauVLSA5nnkfGfCL,42000.00000000,0.10000000,0.00000000,0.00000000,NEW,GTC,LIMIT,BUY,0.00000000,0.00000000,1703913383983,1703913383983,True,1703913383983,0.00000000,EXPIRE_MAKER
4,BTCUSDT,13836308,-1,1Kgn3r0fA1INq47VP3oBWQ,43000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,SELL,0.00000000,0.00000000,1703913569795,1703913738912,True,1703913569795,0.00000000,EXPIRE_MAKER
5,BTCUSDT,13836398,-1,InXWDpncLGcjIwOIPSJQv3,43000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,SELL,0.00000000,0.00000000,1703913588645,1703913724403,True,1703913588645,0.00000000,EXPIRE_MAKER


In [107]:
trades = client.get_my_trades(symbol="BTCUSDT")
trades_df = pd.DataFrame(trades)

In [110]:
trades_df.time = pd.to_datetime(trades_df.time, unit="ms", errors = "coerce")


In [112]:
trades_df

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,2314724,13830784,-1,42066.10000000,0.00951000,400.04861100,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
1,BTCUSDT,2314725,13830784,-1,42066.68000000,0.00797000,335.27143960,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
2,BTCUSDT,2314726,13830784,-1,42066.69000000,0.00607000,255.34480830,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
3,BTCUSDT,2314727,13830784,-1,42066.96000000,0.00690000,290.26202400,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
4,BTCUSDT,2314728,13830784,-1,42067.38000000,0.01035000,435.39738300,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
5,BTCUSDT,2314729,13830784,-1,42067.46000000,0.00892000,375.24174320,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
6,BTCUSDT,2314730,13830784,-1,42068.08000000,0.00666000,280.17341280,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
7,BTCUSDT,2314731,13830784,-1,42068.39000000,0.01118000,470.32460020,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
8,BTCUSDT,2314732,13830784,-1,42068.49000000,0.00999000,420.26421510,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True
9,BTCUSDT,2314733,13830784,-1,42069.10000000,0.00832000,350.01491200,0.00000000,BTC,2023-12-30 05:05:30.605,True,False,True


# First very simple Trading Bot 

In [113]:
# Condition
40210 % 10 

0

In [114]:
from binance import ThreadedWebsocketManager

In [116]:
twm = ThreadedWebsocketManager()
twm.start()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\matei\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\matei\anaconda3\lib\site-packages\binance\threaded_stream.py", line 59, in run
    self._loop.run_until_complete(self.socket_listener())
  File "C:\Users\matei\anaconda3\lib\asyncio\base_events.py", line 623, in run_until_complete
    self._check_running()
  File "C:\Users\matei\anaconda3\lib\asyncio\base_events.py", line 583, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
C:\Users\matei\anaconda3\lib\threading.py:982: RuntimeWarning: coroutine 'ThreadedApiManager.socket_listener' was never awaited
  self._invoke_excepthook(self)


In [120]:
def simple_bot(msg):

    time = pd.to_datetime(msg["E"], unit="ms")
    price = float(msg["c"])

    print("Time: {} | Price: {}".format(time, price))

    if int(price) % 10 == 0:
        order = client.create_order(symbol="BTCUSDT", side="BUY", type="MARKET", quantity=0.1)
        print("\n" + 50 * "-")
        print("Buy {} BTC for {} USDT".format(order["executedQty"], order["cummulativeQuoteQty"]))
        print(50 * "-" + "\n")
 
        twm.stop() # stop defined inside callback

In [ ]:
twm.start_symbol_miniticker_socket(callback = simple_bot, symbol = "BTCUSDT")
twm.join() # required if stop is defined in callback function